In [1]:
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import soundfile as sf

# Load model configuration
config = XttsConfig()
config.load_json('./xtts/config.json')

config.epochs = 1

print('config OK')

# Step 2: Initialize the model
model = Xtts.init_from_config(config)

print('init OK')

# Step 3: Load the pre-trained weights
model.load_checkpoint(config, checkpoint_dir="D:/LanguageModels/xtts/", eval=True)

print('model OK')

# Optional: If you have CUDA installed and want to use GPU, uncomment the line below
# model.cuda()



config OK
init OK


c:\bin\Python309\lib\site-packages\TTS\utils\io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)
GPT2InferenceM

model OK


In [2]:
import json
import torchaudio
import torch

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples

# from TTS.tts.configs import DatasetConfig



def formatter(root_path='', meta_file_train='', ignored_speakers=None):
    # Load JSON dataset
    with open("D:/LanguageModels/audiodata/dutch_numbers_dataset.json", "r", encoding="utf-8") as f:
        dataset = json.load(f)

    print('type of dataset' , type(dataset))    
    print('type of dataset item' , type(dataset[0]))    
    print(' dataset item' , dataset[0])  

    return dataset

dataset_config = BaseDatasetConfig (
    dataset_name = "tts_dataset_getallen",
    path = "D:/LanguageModels/audiodata",
    # root_path = "D:/LanguageModels/audiodata",
    formatter = formatter,
    meta_file_train = "D:/LanguageModels/audiodata/dutch_numbers_dataset.json",
    meta_file_val = "D:/LanguageModels/audiodata/dutch_numbers_dataset.json",
    ignored_speakers = [] ,  # Define an empty list as a placeholder
    language = "Dutch",  # Define language if missing
    meta_file_attn_mask = None ,  # Define an empty list as a placeholder

)

# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, formatter=formatter)

print('train_samples' , train_samples)


# Function to load audio and preprocess dataset
def preprocess(example):
    # Load waveform and sample rate from file
    waveform, sample_rate = torchaudio.load(example["audio_file"])
    
    return {
        "speech_input": waveform,  # Tensor representation of audio
        # "sample_rate": sample_rate,  # Keep track of sample rate
        # "text": example["text"],  # Raw number text
        "text_output": example["spoken_text"]  # Human pronunciation
    }

    # return {
    #     "audio": waveform,  # Tensor representation of audio
    #     "sample_rate": sample_rate,  # Keep track of sample rate
    #     "text": example["text"],  # Raw number text
    #     "spoken_text": example["spoken_text"]  # Human pronunciation
    # }

# # Apply preprocessing to all examples
# processed_dataset = [preprocess(example) for example in dataset]

# # Example: Check first entry
# print(processed_dataset[0])


type of dataset <class 'list'>
type of dataset item <class 'dict'>
 dataset item {'root_path': 'D:/LanguageModels/audiodata', 'audio_file': 'D:/LanguageModels/audiodata/number1.wav', 'text': '1', 'spoken_text': 'een', 'speaker_id': 'speaker1', 'duration': 2.0}
 | > Found 10 files in D:\LanguageModels\audiodata
type of dataset <class 'list'>
type of dataset item <class 'dict'>
 dataset item {'root_path': 'D:/LanguageModels/audiodata', 'audio_file': 'D:/LanguageModels/audiodata/number1.wav', 'text': '1', 'spoken_text': 'een', 'speaker_id': 'speaker1', 'duration': 2.0}
train_samples [{'root_path': 'D:/LanguageModels/audiodata', 'audio_file': 'D:/LanguageModels/audiodata/number1.wav', 'text': '1', 'spoken_text': 'een', 'speaker_id': 'speaker1', 'duration': 2.0, 'language': 'Dutch', 'audio_unique_name': 'tts_dataset_getallen#number1'}, {'root_path': 'D:/LanguageModels/audiodata', 'audio_file': 'D:/LanguageModels/audiodata/number2.wav', 'text': '2', 'spoken_text': 'twee', 'speaker_id': 'spea

In [16]:
from trainer import Trainer

from torch.nn import CrossEntropyLoss

import TTS.tts.configs
print(dir(TTS.tts.configs.xtts_config))
from TTS.tts.configs.xtts_config import XttsConfig

from trainer import TrainerArgs

from trainer import TrainerConfig

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import numpy as np
import librosa
# import importlib
# import ultratb

# # Reload the module after making changes
# importlib.reload(ultratb)


config.output_path = 'D:/LanguageModels/logs'

class CustomDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        print('CustomDataset sample' , self.samples[idx])
        return self.samples[idx]
    

class XTTSWrapper(nn.Module):

    def __init__(self, pretrained_model_name="coqui/XTTS-v2", train_samples=None, batch_size=4):
        super().__init__()

        # Load the pre-trained XTTS model
        self.model = model
        
        self.train_samples = train_samples
        self.batch_size = batch_size

        # Define criterion (loss function)
        self.criterion = nn.CrossEntropyLoss()  # Modify based on task

    def get_criterion(self):
        """Provide the expected method"""
        return self.criterion
    
    def get_data_loader(self, config, assets, is_eval, samples, verbose, num_gpus):
        # wrap the model's data loader
        return self.model.get_data_loader(
                    config=config, assets=assets, is_eval=is_eval, samples=samples, verbose=verbose, num_gpus=num_gpus
                )
    
    def get_train_data_loader(self, config, training_assets, samples, verbose, num_gpus):
        """Return DataLoader for training samples."""
        dataset = CustomDataset(self.train_samples)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
    
    
    def extract_mel_spectrogram(self, audio_file):
        """Convert audio file into Mel spectrogram for XTTS."""
        y, sr = librosa.load(audio_file, sr=24000)  # Load waveform (XTTS uses 24kHz)
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=80)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # Convert to dB

        return mel_spec_db  # Return processed Mel spectrogram    
 
    def format_batch(self, batch):
            """Format batch for XTTS training."""
            print('batch' , batch)

            """Convert audio files to Mel spectrograms and format batch."""
            audio_files = batch["audio_file"]
            audio_features = []
            
            for audio_file in audio_files:
                audio_feature = self.extract_mel_spectrogram(audio_file)  # Convert audio
                audio_features.append(audio_feature)

            batch['audio_features'] = audio_features

            return batch

            return batch
            # texts = [sample["text"] for sample in batch]
            # audio_files = [sample["audio_file"] for sample in batch]
            # speaker_ids = [sample["speaker_id"] for sample in batch]

            # return {
            #     "texts": texts,
            #     "audio_files": audio_files,
            #     "speaker_ids": speaker_ids
            # }

    def format_batch_on_device(self, batch):
            """Format batch for XTTS training."""
            print('batch on device' , batch)

            return batch


    def forward(self, input_text, speaker_embeddings, audio_features):
        """Run XTTS forward pass"""
        return self.model(input_text, speaker_embeddings, audio_features)

    def train_step(self, batch, criterion):
            """Perform a single training step."""
            texts = batch["text"]
            # speaker_embeddings = batch["speaker_embeddings"]
            audio_features = batch["audio_features"]

            print('XTTS expected arguments ' , self.model.forward.__code__.co_varnames)  # Lists expected arguments
            print(self.model.forward.__code__.co_argcount)  # Shows argument count

            print('XTTS functions' , dir(self.model))

            # Forward pass
            output = self.model.train_step(batch, criterion)
            # output = self.model(texts, audio_features)
            # output = self.model(texts, speaker_embeddings, audio_features)

            # Define loss function
            loss = criterion(output, audio_features)  # Modify based on XTTS loss requirements

            return loss


# Initialize the wrapped model
wrapped_model = XTTSWrapper(train_samples=train_samples)


# Load dataset
# train_dataset, eval_dataset = load_tts_samples("dataset/", eval_split=True)
trainerArgs = TrainerArgs()

print('trainerArgs' , dir(trainerArgs))

trainerArgs.log_to_file = False  # Prevents logging to disk


# import logging
# logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler()])


# config = XttsConfig() 

# Define fine-tuning configuration
trainerConfig = TrainerConfig(
    epochs = 3,  # Reduce epochs for fine-tuning
    batch_size = 4,
    save_step = 10000,
    output_path = "D:/LanguageModels/xtts-finetuned"
)


print('model attributes' , dir(model))

# Initialize trainer with pre-trained model
trainer = Trainer(
    model = wrapped_model,
    args = trainerArgs,
    config = config,
    output_path = "D:/LanguageModels/xtts-finetuned",
    train_samples = train_samples,
    eval_samples = eval_samples
)

import logging

class NoLoggingHandler(logging.Handler):
    def emit(self, record):
        pass  # Prevent logs from being written

logging.getLogger().addHandler(NoLoggingHandler())


trainer.criterion = CrossEntropyLoss()



# Start fine-tuning
trainer.fit()


['BaseTTSConfig', 'List', 'XttsArgs', 'XttsAudioConfig', 'XttsConfig', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'dataclass', 'field']
trainerArgs ['_MutableMapping__marker', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__contains__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_initialized', '_is_initialized', '_keytransform', '_validate_contracts', 'best_path', 'check_values', 'clear', 'continue_path', 'copy', 'deserialize', 'deseriali

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 2
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=D:/LanguageModels/logs\run-April-12-2025_05+46AM-40b40a0

 > Model has 466874863 parameters

 > EPOCH: 0/1
 --> D:/LanguageModels/logs\run-April-12-2025_05+46AM-40b40a0

 > TRAINING (2025-04-12 05:46:45) 
 ! Run is removed from D:/LanguageModels/logs\run-April-12-2025_05+46AM-40b40a0
Traceback (most recent call last):
  File "c:\bin\Python309\lib\site-packages\trainer\trainer.py", line 1840, in fit
    self._fit()
  File "c:\bin\Python309\lib\site-packages\trainer\trainer.py", line 1792, in _fit
    self.train_epoch()
  File "c:\bin\Python309\lib\site-packages\trainer\trainer.py", line 1511, in train_epoch
    outputs, _ = self.train_step(batch, batch_n

training assets {}
training samples [{'root_path': 'D:/LanguageModels/audiodata', 'audio_file': 'D:/LanguageModels/audiodata/number1.wav', 'text': '1', 'spoken_text': 'een', 'speaker_id': 'speaker1', 'duration': 2.0, 'language': 'Dutch', 'audio_unique_name': 'tts_dataset_getallen#number1'}, {'root_path': 'D:/LanguageModels/audiodata', 'audio_file': 'D:/LanguageModels/audiodata/number2.wav', 'text': '2', 'spoken_text': 'twee', 'speaker_id': 'speaker1', 'duration': 2.0, 'language': 'Dutch', 'audio_unique_name': 'tts_dataset_getallen#number2'}, {'root_path': 'D:/LanguageModels/audiodata', 'audio_file': 'D:/LanguageModels/audiodata/number3.wav', 'text': '3', 'spoken_text': 'drie', 'speaker_id': 'speaker1', 'duration': 2.0, 'language': 'Dutch', 'audio_unique_name': 'tts_dataset_getallen#number3'}, {'root_path': 'D:/LanguageModels/audiodata', 'audio_file': 'D:/LanguageModels/audiodata/number4.wav', 'text': '4', 'spoken_text': 'vier', 'speaker_id': 'speaker1', 'duration': 2.0, 'language': 'Du

AssertionError: 

In [ ]:
model.save_pretrained("D:/LanguageModels/xtts-getallen")
